In [1]:
!pip install torch imgaug


     |████████████████████████████████| 887.4 MB 2.1 kB/s eta 0:00:013  |                                | 3.0 MB 2.3 MB/s eta 0:06:25     |█▍                              | 39.1 MB 16.8 MB/s eta 0:00:51     |██▏                             | 60.4 MB 12.7 MB/s eta 0:01:06     |██▎                             | 62.8 MB 23.8 MB/s eta 0:00:35     |███                             | 80.1 MB 19.5 MB/s eta 0:00:42     |███▏                            | 88.6 MB 19.5 MB/s eta 0:00:41     |███▍                            | 92.6 MB 19.5 MB/s eta 0:00:41     |██████▊                         | 187.3 MB 4.3 MB/s eta 0:02:44     |███████▌                        | 207.4 MB 19.3 MB/s eta 0:00:36     |███████▋                        | 211.7 MB 19.3 MB/s eta 0:00:36     |██████████                      | 277.6 MB 16.0 MB/s eta 0:00:39     |██████████▊                     | 296.6 MB 32.9 MB/s eta 0:00:18     |███████████▉                    | 327.8 MB 16.6 MB/s eta 0:00:34     |████████████▋              

In [3]:
# Import required modules
import os
import subprocess
import zipfile
import numpy as np
import torch
import uptrain

# Import custom helper functions
from helper_files import read_json, KpsDataset
from helper_files import body_length_signal, plot_all_cluster

# Importing read_json, KpsDataset, body_length_signal, and plot_all_cluster functions from helper_files module 

In [4]:
# This code downloads the example dataset used for training the model.
# The data is stored in a zip file and is hosted on an Amazon S3 bucket.

# Setting the directory where the data will be stored
data_dir = "data"

# URL for the data zip file
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/data.zip"

# File path for the training data
orig_training_file = 'data/training_data.json'

# Check if the data directory exists
if not os.path.exists(data_dir):
    print("Starting to download example dataset...")

    # Try to install Wget on MacOS
    try:
        wget_installed_ok = subprocess.call("brew install wget", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
        print("Successfully installed wget")
    except:
        dummy = 1
    
    # Try to download the data zip file and extract the data
    try:
        if not os.path.exists("data.zip"):
            file_downloaded_ok = subprocess.call("wget " + remote_url, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
            print("Data downloaded")
        with zipfile.ZipFile("data.zip", 'r') as zip_ref:
            zip_ref.extractall("./")
        print("Prepared Example Dataset")
        os.remove("data.zip")
    except Exception as e:
        print(e)
        print("Could not load training data")
        print("Please follow following steps to manually download data")
        print("Step 1: Paste the link https://oodles-dev-training-data.s3.amazonaws.com/data.zip in your browser")
        print("Step 2: Once the zip file is downloaded, unzip it here (i.e. YOUR_LOC/uptrain/examples/1_orientation_classification/")
else:
    print("Example dataset already present")


Starting to download example dataset...
Data downloaded
Prepared Example Dataset


In [5]:
# Training data
training_file = 'data/training_data.json'

# A testing dataset to evaluate model performance
golden_testing_file = 'data/golden_testing_data.json'

# The data-points which the models sees in production
real_world_test_cases = 'data/real_world_testing_data.json'

# To annotate the collected data points, we extract the ground truth from a master annotation file 
# (we can also import from any other annotation pipelines such as an annotation job on Mechanical turk).
annotation_args = {'master_file': 'data/master_annotation_data.json'}

# Let's visualize some of the training examples
training_data = read_json(training_file, dataframe=True)
training_data.head()

,id,gt,Nose_X,Nose_Y,Left_Eye_X,Left_Eye_Y,Right_Eye_X,Right_Eye_Y,Left_Ear_X,Left_Ear_Y,...,Right_Hip_X,Right_Hip_Y,Left_Knee_X,Left_Knee_Y,Right_Knee_X,Right_Knee_Y,Left_Ankle_X,Left_Ankle_Y,Right_Ankle_X,Right_Ankle_Y
0,12002202098,1,536.832198,175.195562,275.183835,299.200492,321.438219,53.586762,438.831630,196.469141,...,355.320493,352.771307,282.603256,245.372110,-71.727270,148.854694,377.204408,118.812093,368.630415,32.683708
1,11004300163,1,127.299427,213.304615,120.006179,213.098156,119.749502,220.877945,123.240242,213.496218,...,282.739062,217.173672,331.887746,122.399542,369.934428,153.331971,407.764187,54.115847,373.738996,228.353284
2,18100208242,0,342.735809,75.173884,347.639763,69.066162,337.646757,69.463328,354.332650,70.183796,...,327.693125,175.678139,373.687116,236.403840,330.044019,238.719585,383.867725,289.895923,335.013380,292.018822
3,18100204340,0,335.262419,161.525842,340.358734,155.666068,329.570863,154.843509,347.034712,154.827196,...,316.727216,218.696415,375.741803,250.993843,285.306140,252.504808,384.364024,298.382275,275.523553,298.044853
4,12100104261,0,255.488160,62.442478,256.765451,58.603055,251.489867,58.323107,255.327531,55.634296,...,235.763698,136.902009,277.832912,177.679435,229.566259,191.052507,284.397434,213.477405,224.158806,239.154790
